In [5]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Parameters
tickers = ['EURUSD', 'USDJPY', 'GBPUSD', 'AUDUSD', 'USDCAD']
buckets = ['ON', '1W', '1M', '3M', '6M']  # Reduced to 5 tenors
num_days = 40  # Generate data for the last 30 days

# Generate daily dates
base_date = datetime.today().date()
dates = [base_date - timedelta(days=i) for i in range(num_days)]

# Generate data
data = []
for ticker in tickers:
    for date in dates:
        for bucket in buckets:
            volume = round(random.uniform(1e6, 1e8), 2)  # Random volume
            data.append({
                'ticker': ticker,
                'bucket': bucket,
                'volume': volume,
                'date': date
            })

# Create DataFrame
FX_Swap_df = pd.DataFrame(data)

# Display sample
FX_Swap_df


,ticker,bucket,volume,date
0,EURUSD,ON,81731585.92,2025-07-04
1,EURUSD,1W,98855429.02,2025-07-04
2,EURUSD,1M,86008409.71,2025-07-04
3,EURUSD,3M,37144288.95,2025-07-04
4,EURUSD,6M,9368139.21,2025-07-04
...,...,...,...,...
995,USDCAD,ON,43181626.08,2025-05-26
996,USDCAD,1W,67870629.43,2025-05-26
997,USDCAD,1M,37903171.47,2025-05-26
998,USDCAD,3M,83672894.37,2025-05-26


In [8]:
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Sample data creation (if FX_Swap_df doesn't exist)
try:
    FX_Swap_df
except NameError:
    np.random.seed(42)
    tickers = ['EURUSD', 'USDJPY', 'GBPUSD', 'AUDUSD', 'USDCAD', 'NZDUSD']
    buckets = ['ON', '1W', '1M', '3M', '6M', '1Y', '2Y']
    dates = pd.date_range(end=datetime.today(), periods=90).date
    
    data = []
    for date in dates:
        for ticker in tickers:
            for bucket in buckets:
                base_vol = np.random.randint(100, 500)
                trend = (date - dates[0]).days / 10
                seasonal = np.sin(2 * np.pi * (date - dates[0]).days / 30) * 50
                data.append({
                    'ticker': ticker,
                    'bucket': bucket,
                    'volume': int(base_vol + trend + seasonal + np.random.normal(0, 20)),
                    'date': date
                })
    
    FX_Swap_df = pd.DataFrame(data)

# Convert date column to datetime
FX_Swap_df['date'] = pd.to_datetime(FX_Swap_df['date'])
max_date = FX_Swap_df['date'].max()

# Initialize the Dash app
app = dash.Dash(__name__)
server = app.server

# Get unique tickers and buckets for dropdowns
all_tickers = sorted(FX_Swap_df['ticker'].unique())
all_buckets = sorted(FX_Swap_df['bucket'].unique())

app.layout = html.Div([
    html.Div([
        html.H1("FX Swap Volume Analysis Dashboard", style={'textAlign': 'center'}),
        
        html.Div([
            html.Div([
                html.Label("Select Tickers:", style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='ticker-selector',
                    options=[{'label': t, 'value': t} for t in all_tickers],
                    value=all_tickers[:3],
                    multi=True,
                    style={'width': '100%'}
                ),
            ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
            
            html.Div([
                html.Label("Time Window (days from most recent date):", style={'fontWeight': 'bold'}),
                dcc.Slider(
                    id='time-window-slider',
                    min=1,
                    max=180,
                    value=30,
                    marks={i: str(i) for i in range(0, 181, 30)},
                    step=1,
                    tooltip={"placement": "bottom", "always_visible": True}
                )
            ], style={'width': '65%', 'display': 'inline-block', 'padding': '10px'}),
            
            html.Div([
                html.Label("Additional Options:", style={'fontWeight': 'bold'}),
                dcc.Checklist(
                    id='display-options',
                    options=[
                        {'label': ' Show Trend Lines', 'value': 'show_trend'},
                        {'label': ' Normalize Scales', 'value': 'normalize'},
                        {'label': ' Show Data Table', 'value': 'show_table'}
                    ],
                    value=[],
                    labelStyle={'display': 'inline-block', 'marginRight': '20px'}
                )
            ], style={'width': '100%', 'padding': '10px'}),
            
            # <-- New button -->
            html.Div([
                html.Button("Show Full Data", id="show-full-data-btn", n_clicks=0)
            ], style={'padding': '10px'})
        ], style={'border': '1px solid #ddd', 'borderRadius': '5px', 'padding': '10px', 'margin': '10px 0'}),
        
        dcc.Graph(id='volume-matrix-plot'),
        
        # Container for either the summary table or the full table
        html.Div(id='data-table-container', style={'margin': '20px 0'})
    ], style={'padding': '20px'})
])

@app.callback(
    Output('data-table-container', 'children'),
    [
        Input('ticker-selector', 'value'),
        Input('time-window-slider', 'value'),
        Input('display-options', 'value'),
        Input('show-full-data-btn', 'n_clicks')
    ]
)
def update_table(selected_tickers, time_window_days, display_options, full_data_clicks):
    ctx = dash.callback_context
    # If the full-data button was clicked most recently, show full DF
    if ctx.triggered and ctx.triggered[0]['prop_id'].startswith('show-full-data-btn'):
        return dash_table.DataTable(
            id='full-data-table',
            columns=[{"name": i, "id": i} for i in FX_Swap_df.columns],
            data=FX_Swap_df.to_dict('records'),
            page_size=20,
            style_table={'overflowX': 'auto', 'maxHeight': '500px'},
            style_cell={'minWidth': '100px', 'whiteSpace': 'normal'},
            style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}
        )
    
    # Otherwise, if 'show_table' is checked, show summary stats
    if 'show_table' in display_options:
        cutoff_date = max_date - timedelta(days=time_window_days)
        filtered_df = FX_Swap_df[
            (FX_Swap_df['date'] >= cutoff_date) &
            (FX_Swap_df['ticker'].isin(selected_tickers))
        ]
        if not filtered_df.empty:
            summary_df = filtered_df.groupby(['ticker', 'bucket'])['volume'] \
                                    .agg(['mean', 'min', 'max', 'std']) \
                                    .reset_index().round(2)
            return html.Div([
                html.H3("Summary Statistics"),
                dash_table.DataTable(
                    id='data-table',
                    columns=[{"name": i, "id": i} for i in summary_df.columns],
                    data=summary_df.to_dict('records'),
                    style_table={'overflowX': 'auto'},
                    style_cell={
                        'minWidth': '100px', 'width': '150px', 'maxWidth': '200px',
                        'whiteSpace': 'normal'
                    },
                    style_header={
                        'backgroundColor': 'rgb(230, 230, 230)',
                        'fontWeight': 'bold'
                    },
                    style_data_conditional=[
                        {
                            'if': {'row_index': 'odd'},
                            'backgroundColor': 'rgb(248, 248, 248)'
                        }
                    ]
                )
            ])
    # If neither, clear output
    return None

@app.callback(
    Output('volume-matrix-plot', 'figure'),
    [
        Input('ticker-selector', 'value'),
        Input('time-window-slider', 'value'),
        Input('display-options', 'value')
    ]
)
def update_plot(selected_tickers, time_window_days, display_options):
    cutoff_date = max_date - timedelta(days=time_window_days)
    filtered_df = FX_Swap_df[
        (FX_Swap_df['date'] >= cutoff_date) & 
        (FX_Swap_df['ticker'].isin(selected_tickers))
    ].sort_values(['ticker', 'bucket', 'date'])
    
    buckets = sorted(filtered_df['bucket'].unique())
    fig = make_subplots(
        rows=len(buckets), 
        cols=len(selected_tickers),
        subplot_titles=selected_tickers,
        shared_yaxes='normalize' in display_options,
        vertical_spacing=0.05,
        horizontal_spacing=0.05
    )
    
    # Row labels
    for i, bucket in enumerate(buckets):
        fig.add_annotation(
            x=-0.07, y=1 - (i + 0.5)/len(buckets),
            xref="paper", yref="paper",
            text=bucket, showarrow=False, textangle=-90,
            font=dict(size=12), bgcolor='rgba(255,255,255,0.7)'
        )
    
    # Plot bars & trend
    for col_idx, ticker in enumerate(selected_tickers, start=1):
        for row_idx, bucket in enumerate(buckets, start=1):
            subset = filtered_df[
                (filtered_df['ticker'] == ticker) &
                (filtered_df['bucket'] == bucket)
            ]
            fig.add_trace(
                go.Bar(x=subset['date'], y=subset['volume'],
                       showlegend=False, marker_color='#1f77b4', opacity=0.7),
                row=row_idx, col=col_idx
            )
            if 'show_trend' in display_options and len(subset) > 1:
                fig.add_trace(
                    go.Scatter(
                        x=subset['date'],
                        y=subset['volume'].rolling(7, min_periods=1).mean(),
                        mode='lines', line=dict(color='red', width=2),
                        showlegend=False
                    ), row=row_idx, col=col_idx
                )
            fig.update_xaxes(
                title_text="Date" if row_idx == len(buckets) else "",
                row=row_idx, col=col_idx, tickformat="%b %d", tickangle=45
            )
            if col_idx == 1:
                fig.update_yaxes(title_text="Volume", row=row_idx, col=col_idx)
    
    fig.update_layout(
        height=200 * len(buckets),
        width=300 * max(len(selected_tickers), 1),
        title_text=f"FX Swap Volume Evolution (Last {time_window_days} Days)",
        margin=dict(l=100, r=50, b=100, t=50),
        hovermode="closest",
        plot_bgcolor='rgba(240,240,240,0.5)'
    )
    return fig

if __name__ == '__main__':
    app.run(debug=True, port=8051)


In [12]:
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Sample data creation
np.random.seed(42)
tickers = ['EURUSD', 'USDJPY', 'GBPUSD', 'AUDUSD', 'USDCAD', 'NZDUSD']
buckets = ['ON', '1W', '1M', '3M', '6M', '1Y', '2Y']
dates = pd.date_range(end=datetime.today(), periods=90).date

data = []
for date in dates:
    for ticker in tickers:
        for bucket in buckets:
            # Create some trends in the data
            base_vol = np.random.randint(100, 500)
            trend = (date - dates[0]).days / 10
            seasonal = np.sin(2 * np.pi * (date - dates[0]).days / 30) * 50
            data.append({
                'ticker': ticker,
                'bucket': bucket,
                'volume': int(base_vol + trend + seasonal + np.random.normal(0, 20)),
                'date': date
            })

FX_Swap_df = pd.DataFrame(data)

# Convert date column to datetime
FX_Swap_df['date'] = pd.to_datetime(FX_Swap_df['date'])
max_date = FX_Swap_df['date'].max()

# Initialize the Dash app
app = dash.Dash(__name__)
server = app.server

# Get unique tickers and buckets for dropdowns
all_tickers = sorted(FX_Swap_df['ticker'].unique())
all_buckets = sorted(FX_Swap_df['bucket'].unique())

app.layout = html.Div([
    html.Div([
        html.H1("FX Swap Volume Analysis Dashboard", style={'textAlign': 'center'}),
        
        html.Div([
            html.Div([
                html.Label("Select Tickers:", style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='ticker-selector',
                    options=[{'label': t, 'value': t} for t in all_tickers],
                    value=all_tickers[:3],  # Default to first 3 tickers
                    multi=True,
                    style={'width': '100%'}
                ),
            ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
            
            html.Div([
                html.Label("Time Window (days from most recent date):", style={'fontWeight': 'bold'}),
                dcc.Slider(
                    id='time-window-slider',
                    min=1,
                    max=180,
                    value=30,
                    marks={i: str(i) for i in range(0, 181, 30)},
                    step=1,
                    tooltip={"placement": "bottom", "always_visible": True}
                )
            ], style={'width': '65%', 'display': 'inline-block', 'padding': '10px'}),
            
            html.Div([
                html.Label("Additional Options:", style={'fontWeight': 'bold'}),
                dcc.Checklist(
                    id='display-options',
                    options=[
                        {'label': ' Show Trend Lines', 'value': 'show_trend'},
                        {'label': ' Normalize Scales', 'value': 'normalize'},
                        {'label': ' Show Data Table', 'value': 'show_table'}
                    ],
                    value=[],
                    labelStyle={'display': 'inline-block', 'marginRight': '20px'}
                )
            ], style={'width': '100%', 'padding': '10px'}),
        ], style={'border': '1px solid #ddd', 'borderRadius': '5px', 'padding': '10px', 'margin': '10px 0'}),
        
        dcc.Tabs([
            dcc.Tab(label='Matrix Barchart', children=[
                dcc.Graph(id='volume-matrix-plot'),
                html.Div(id='data-table-container', style={'margin': '20px 0'})
            ]),
            dcc.Tab(label='Full Dataset Visualization', children=[
                dcc.Graph(id='full-dataset-heatmap'),
                dcc.Graph(id='volume-trend-plot')
            ])
        ]),
    ], style={'padding': '20px'})
])

@app.callback(
    [Output('volume-matrix-plot', 'figure'),
     Output('data-table-container', 'children')],
    [Input('ticker-selector', 'value'),
     Input('time-window-slider', 'value'),
     Input('display-options', 'value')]
)
def update_plots(selected_tickers, time_window_days, display_options):
    # Filter data based on selections
    cutoff_date = max_date - timedelta(days=time_window_days)
    filtered_df = FX_Swap_df[
        (FX_Swap_df['date'] >= cutoff_date) & 
        (FX_Swap_df['ticker'].isin(selected_tickers))
    ].sort_values(['ticker', 'bucket', 'date'])
    
    # Get unique buckets from filtered data
    buckets = sorted(filtered_df['bucket'].unique())
    
    # Create subplots matrix
    fig = make_subplots(
        rows=len(buckets), 
        cols=len(selected_tickers),
        subplot_titles=selected_tickers,  # Column titles
        shared_yaxes='normalize' in display_options,
        vertical_spacing=0.05,
        horizontal_spacing=0.05
    )
    
    # Add row labels (buckets)
    for i, bucket in enumerate(buckets):
        fig.add_annotation(
            x=-0.07,
            y=1 - (i + 0.5)/len(buckets),
            xref="paper",
            yref="paper",
            text=bucket,
            showarrow=False,
            textangle=-90,
            font=dict(size=12, color='black'),
            bgcolor='rgba(255,255,255,0.7)'
        )
    
    # Populate each subplot
    for col_idx, ticker in enumerate(selected_tickers, start=1):
        for row_idx, bucket in enumerate(buckets, start=1):
            subset = filtered_df[
                (filtered_df['ticker'] == ticker) & 
                (filtered_df['bucket'] == bucket)
            ]
            
            # Add bar chart
            fig.add_trace(
                go.Bar(
                    x=subset['date'],
                    y=subset['volume'],
                    name=f"{ticker} {bucket}",
                    showlegend=False,
                    marker_color='#1f77b4',
                    opacity=0.7
                ),
                row=row_idx,
                col=col_idx
            )
            
            # Add trend line if selected
            if 'show_trend' in display_options and len(subset) > 1:
                fig.add_trace(
                    go.Scatter(
                        x=subset['date'],
                        y=subset['volume'].rolling(7, min_periods=1).mean(),
                        mode='lines',
                        line=dict(color='red', width=2),
                        name='Trend',
                        showlegend=False
                    ),
                    row=row_idx,
                    col=col_idx
                )
            
            # Update axes
            fig.update_xaxes(
                title_text="Date" if row_idx == len(buckets) else "",
                row=row_idx,
                col=col_idx,
                tickformat="%b %d",
                tickangle=45
            )
            
            if col_idx == 1:
                fig.update_yaxes(
                    title_text="Volume",
                    row=row_idx,
                    col=col_idx
                )
    
    # Update layout
    fig.update_layout(
        height=200 * len(buckets),
        width=300 * max(len(selected_tickers), 1),
        title_text=f"FX Swap Volume Evolution (Last {time_window_days} Days)",
        margin=dict(l=100, r=50, b=100, t=50),
        hovermode="closest",
        plot_bgcolor='rgba(240,240,240,0.5)'
    )
    
    # Create data table if selected
    table = None
    if 'show_table' in display_options and len(filtered_df) > 0:
        summary_df = filtered_df.groupby(['ticker', 'bucket'])['volume'].agg(['mean', 'min', 'max', 'std'])
        summary_df = summary_df.reset_index().round(2)
        
        table = dash_table.DataTable(
            id='data-table',
            columns=[{"name": i, "id": i} for i in summary_df.columns],
            data=summary_df.to_dict('records'),
            style_table={'overflowX': 'auto'},
            style_cell={
                'minWidth': '100px', 'width': '150px', 'maxWidth': '200px',
                'whiteSpace': 'normal'
            },
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                }
            ]
        )
    
    return fig, [html.H3("Summary Statistics"), table] if table else None

@app.callback(
    [Output('full-dataset-heatmap', 'figure'),
     Output('volume-trend-plot', 'figure')],
    [Input('ticker-selector', 'value')]
)
def update_full_dataset_visualizations(selected_tickers):
    # Create a pivot table for the heatmap
    pivot_df = FX_Swap_df.groupby(['ticker', 'bucket'])['volume'].mean().reset_index()
    pivot_df = pivot_df.pivot(index='bucket', columns='ticker', values='volume')
    
    # Heatmap of average volumes
    heatmap_fig = go.Figure(data=go.Heatmap(
        z=pivot_df.values,
        x=pivot_df.columns,
        y=pivot_df.index,
        colorscale='Viridis',
        hoverongaps=False,
        colorbar=dict(title="Average Volume")
    ))
    
    heatmap_fig.update_layout(
        title="Average FX Swap Volume by Currency Pair and Tenor",
        xaxis_title="Currency Pair",
        yaxis_title="Tenor",
        height=500,
        margin=dict(l=100, r=50, b=100, t=50)
    )
    
    # Line plot showing trends over time
    if selected_tickers:
        trend_df = FX_Swap_df[FX_Swap_df['ticker'].isin(selected_tickers)]
        trend_fig = px.line(
            trend_df.groupby(['date', 'ticker'])['volume'].mean().reset_index(),
            x='date',
            y='volume',
            color='ticker',
            title="FX Swap Volume Trends Over Time",
            labels={'volume': 'Average Volume', 'date': 'Date'}
        )
        
        trend_fig.update_layout(
            height=500,
            hovermode="x unified",
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )
    else:
        trend_fig = go.Figure()
        trend_fig.update_layout(
            title="Select tickers to view trends",
            height=500
        )
    
    return heatmap_fig, trend_fig

if __name__ == '__main__':
    app.run(debug=True, port=8051)

In [13]:
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Initialize the Dash app
app = dash.Dash(__name__)
server = app.server

def create_sample_data():
    """Generate sample FX swap data"""
    np.random.seed(42)
    tickers = ['EURUSD', 'USDJPY', 'GBPUSD', 'AUDUSD', 'USDCAD', 'NZDUSD']
    buckets = ['ON', '1W', '1M', '3M', '6M', '1Y', '2Y']
    dates = pd.date_range(end=datetime.today(), periods=90).date
    
    data = []
    for date in dates:
        for ticker in tickers:
            for bucket in buckets:
                base_vol = np.random.randint(100, 500)
                trend = (date - dates[0]).days / 10
                seasonal = np.sin(2 * np.pi * (date - dates[0]).days / 30) * 50
                data.append({
                    'ticker': ticker,
                    'bucket': bucket,
                    'volume': int(base_vol + trend + seasonal + np.random.normal(0, 20)),
                    'date': date
                })
    
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'])
    return df

FX_Swap_df = create_sample_data()
max_date = FX_Swap_df['date'].max()
all_tickers = sorted(FX_Swap_df['ticker'].unique())
all_buckets = sorted(FX_Swap_df['bucket'].unique())

def construct_layout():
    """Build the complete application layout"""
    return html.Div([
        html.Div([
            html.H1("FX Swap Volume Analysis Dashboard", style={'textAlign': 'center'}),
            
            html.Div([
                html.Div([
                    html.Label("Select Tickers:", style={'fontWeight': 'bold'}),
                    dcc.Dropdown(
                        id='ticker-selector',
                        options=[{'label': t, 'value': t} for t in all_tickers],
                        value=all_tickers[:3],
                        multi=True,
                        style={'width': '100%'}
                    ),
                ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
                
                html.Div([
                    html.Label("Time Window (days from most recent date):", style={'fontWeight': 'bold'}),
                    dcc.Slider(
                        id='time-window-slider',
                        min=1,
                        max=180,
                        value=30,
                        marks={i: str(i) for i in range(0, 181, 30)},
                        step=1,
                        tooltip={"placement": "bottom", "always_visible": True}
                    )
                ], style={'width': '65%', 'display': 'inline-block', 'padding': '10px'}),
                
                html.Div([
                    html.Label("Additional Options:", style={'fontWeight': 'bold'}),
                    dcc.Checklist(
                        id='display-options',
                        options=[
                            {'label': ' Show Trend Lines', 'value': 'show_trend'},
                            {'label': ' Normalize Scales', 'value': 'normalize'},
                            {'label': ' Show Data Table', 'value': 'show_table'}
                        ],
                        value=[],
                        labelStyle={'display': 'inline-block', 'marginRight': '20px'}
                    )
                ], style={'width': '100%', 'padding': '10px'}),
            ], style={'border': '1px solid #ddd', 'borderRadius': '5px', 'padding': '10px', 'margin': '10px 0'}),
            
            dcc.Tabs([
                dcc.Tab(label='Matrix Barchart', children=[
                    dcc.Graph(id='volume-matrix-plot'),
                    html.Div(id='data-table-container', style={'margin': '20px 0'})
                ]),
                dcc.Tab(label='Full Dataset Visualization', children=[
                    dcc.Graph(id='full-dataset-heatmap'),
                    dcc.Graph(id='volume-trend-plot')
                ])
            ]),
        ], style={'padding': '20px'})
    ])

def register_callbacks():
    """Register all application callbacks"""
    
    @app.callback(
        [Output('volume-matrix-plot', 'figure'),
         Output('data-table-container', 'children')],
        [Input('ticker-selector', 'value'),
         Input('time-window-slider', 'value'),
         Input('display-options', 'value')]
    )
    def update_plots(selected_tickers, time_window_days, display_options):
        # Filter data based on selections
        cutoff_date = max_date - timedelta(days=time_window_days)
        filtered_df = FX_Swap_df[
            (FX_Swap_df['date'] >= cutoff_date) & 
            (FX_Swap_df['ticker'].isin(selected_tickers))
        ].sort_values(['ticker', 'bucket', 'date'])
        
        buckets = sorted(filtered_df['bucket'].unique())
        
        # Create subplots matrix
        fig = make_subplots(
            rows=len(buckets), 
            cols=len(selected_tickers),
            subplot_titles=selected_tickers,
            shared_yaxes='normalize' in display_options,
            vertical_spacing=0.05,
            horizontal_spacing=0.05
        )
        
        # Add row labels (buckets)
        for i, bucket in enumerate(buckets):
            fig.add_annotation(
                x=-0.07,
                y=1 - (i + 0.5)/len(buckets),
                xref="paper",
                yref="paper",
                text=bucket,
                showarrow=False,
                textangle=-90,
                font=dict(size=12, color='black'),
                bgcolor='rgba(255,255,255,0.7)'
            )
        
        # Populate each subplot
        for col_idx, ticker in enumerate(selected_tickers, start=1):
            for row_idx, bucket in enumerate(buckets, start=1):
                subset = filtered_df[
                    (filtered_df['ticker'] == ticker) & 
                    (filtered_df['bucket'] == bucket)
                ]
                
                # Add bar chart
                fig.add_trace(
                    go.Bar(
                        x=subset['date'],
                        y=subset['volume'],
                        name=f"{ticker} {bucket}",
                        showlegend=False,
                        marker_color='#1f77b4',
                        opacity=0.7
                    ),
                    row=row_idx,
                    col=col_idx
                )
                
                # Add trend line if selected
                if 'show_trend' in display_options and len(subset) > 1:
                    fig.add_trace(
                        go.Scatter(
                            x=subset['date'],
                            y=subset['volume'].rolling(7, min_periods=1).mean(),
                            mode='lines',
                            line=dict(color='red', width=2),
                            name='Trend',
                            showlegend=False
                        ),
                        row=row_idx,
                        col=col_idx
                    )
                
                # Update axes
                fig.update_xaxes(
                    title_text="Date" if row_idx == len(buckets) else "",
                    row=row_idx,
                    col=col_idx,
                    tickformat="%b %d",
                    tickangle=45
                )
                
                if col_idx == 1:
                    fig.update_yaxes(
                        title_text="Volume",
                        row=row_idx,
                        col=col_idx
                    )
        
        # Update layout
        fig.update_layout(
            height=200 * len(buckets),
            width=300 * max(len(selected_tickers), 1),
            title_text=f"FX Swap Volume Evolution (Last {time_window_days} Days)",
            margin=dict(l=100, r=50, b=100, t=50),
            hovermode="closest",
            plot_bgcolor='rgba(240,240,240,0.5)'
        )
        
        # Create data table if selected
        table = None
        if 'show_table' in display_options and len(filtered_df) > 0:
            summary_df = filtered_df.groupby(['ticker', 'bucket'])['volume'].agg(['mean', 'min', 'max', 'std'])
            summary_df = summary_df.reset_index().round(2)
            
            table = dash_table.DataTable(
                id='data-table',
                columns=[{"name": i, "id": i} for i in summary_df.columns],
                data=summary_df.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_cell={
                    'minWidth': '100px', 'width': '150px', 'maxWidth': '200px',
                    'whiteSpace': 'normal'
                },
                style_header={
                    'backgroundColor': 'rgb(230, 230, 230)',
                    'fontWeight': 'bold'
                },
                style_data_conditional=[
                    {
                        'if': {'row_index': 'odd'},
                        'backgroundColor': 'rgb(248, 248, 248)'
                    }
                ]
            )
        
        return fig, [html.H3("Summary Statistics"), table] if table else None

    @app.callback(
        [Output('full-dataset-heatmap', 'figure'),
         Output('volume-trend-plot', 'figure')],
        [Input('ticker-selector', 'value')]
    )
    def update_full_dataset_visualizations(selected_tickers):
        # Create a pivot table for the heatmap
        pivot_df = FX_Swap_df.groupby(['ticker', 'bucket'])['volume'].mean().reset_index()
        pivot_df = pivot_df.pivot(index='bucket', columns='ticker', values='volume')
        
        # Heatmap of average volumes
        heatmap_fig = go.Figure(data=go.Heatmap(
            z=pivot_df.values,
            x=pivot_df.columns,
            y=pivot_df.index,
            colorscale='Viridis',
            hoverongaps=False,
            colorbar=dict(title="Average Volume")
        ))
        
        heatmap_fig.update_layout(
            title="Average FX Swap Volume by Currency Pair and Tenor",
            xaxis_title="Currency Pair",
            yaxis_title="Tenor",
            height=500,
            margin=dict(l=100, r=50, b=100, t=50)
        )
        
        # Line plot showing trends over time
        if selected_tickers:
            trend_df = FX_Swap_df[FX_Swap_df['ticker'].isin(selected_tickers)]
            trend_fig = px.line(
                trend_df.groupby(['date', 'ticker'])['volume'].mean().reset_index(),
                x='date',
                y='volume',
                color='ticker',
                title="FX Swap Volume Trends Over Time",
                labels={'volume': 'Average Volume', 'date': 'Date'}
            )
            
            trend_fig.update_layout(
                height=500,
                hovermode="x unified",
                legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1
                )
            )
        else:
            trend_fig = go.Figure()
            trend_fig.update_layout(
                title="Select tickers to view trends",
                height=500
            )
        
        return heatmap_fig, trend_fig

# Set up the application
app.layout = construct_layout()
register_callbacks()

if __name__ == '__main__':
    app.run(debug=True, port=8051)

In [14]:
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# ─── Sample data creation ──────────────────────────────────────────────────────
try:
    FX_Swap_df
except NameError:
    np.random.seed(42)
    tickers = ['EURUSD', 'USDJPY', 'GBPUSD', 'AUDUSD', 'USDCAD', 'NZDUSD']
    buckets = ['ON', '1W', '1M', '3M', '6M', '1Y', '2Y']
    dates = pd.date_range(end=datetime.today(), periods=90).date

    data = []
    for date in dates:
        for ticker in tickers:
            for bucket in buckets:
                base_vol = np.random.randint(100, 500)
                trend = (date - dates[0]).days / 10
                seasonal = np.sin(2 * np.pi * (date - dates[0]).days / 30) * 50
                data.append({
                    'ticker': ticker,
                    'bucket': bucket,
                    'volume': int(base_vol + trend + seasonal + np.random.normal(0, 20)),
                    'date': date
                })

    FX_Swap_df = pd.DataFrame(data)

FX_Swap_df['date'] = pd.to_datetime(FX_Swap_df['date'])
max_date = FX_Swap_df['date'].max()

# ─── App initialization ───────────────────────────────────────────────────────
app = dash.Dash(__name__)
server = app.server  # for gunicorn, etc.

# ─── Layout construction ──────────────────────────────────────────────────────
def construct_layout():
    all_tickers = sorted(FX_Swap_df['ticker'].unique())

    return html.Div([
        html.Div([
            html.H1("FX Swap Volume Analysis Dashboard", style={'textAlign': 'center'}),

            html.Div([
                html.Div([
                    html.Label("Select Tickers:", style={'fontWeight': 'bold'}),
                    dcc.Dropdown(
                        id='ticker-selector',
                        options=[{'label': t, 'value': t} for t in all_tickers],
                        value=all_tickers[:3],
                        multi=True,
                        style={'width': '100%'}
                    ),
                ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),

                html.Div([
                    html.Label("Time Window (days from most recent date):", style={'fontWeight': 'bold'}),
                    dcc.Slider(
                        id='time-window-slider',
                        min=1,
                        max=180,
                        value=30,
                        marks={i: str(i) for i in range(0, 181, 30)},
                        step=1,
                        tooltip={"placement": "bottom", "always_visible": True}
                    )
                ], style={'width': '65%', 'display': 'inline-block', 'padding': '10px'}),

                html.Div([
                    html.Label("Additional Options:", style={'fontWeight': 'bold'}),
                    dcc.Checklist(
                        id='display-options',
                        options=[
                            {'label': ' Show Trend Lines', 'value': 'show_trend'},
                            {'label': ' Normalize Scales', 'value': 'normalize'},
                            {'label': ' Show Data Table', 'value': 'show_table'}
                        ],
                        value=[],
                        labelStyle={'display': 'inline-block', 'marginRight': '20px'}
                    )
                ], style={'width': '100%', 'padding': '10px'}),

                html.Div([
                    html.Button("Show Full Data", id="show-full-data-btn", n_clicks=0)
                ], style={'padding': '10px'})
            ], style={
                'border': '1px solid #ddd',
                'borderRadius': '5px',
                'padding': '10px',
                'margin': '10px 0'
            }),

            dcc.Graph(id='volume-matrix-plot'),

            html.Div(id='data-table-container', style={'margin': '20px 0'})
        ], style={'padding': '20px'})
    ])

# ─── Callback registration ────────────────────────────────────────────────────
def register_callbacks(app):

    @app.callback(
        Output('data-table-container', 'children'),
        [
            Input('ticker-selector', 'value'),
            Input('time-window-slider', 'value'),
            Input('display-options', 'value'),
            Input('show-full-data-btn', 'n_clicks')
        ]
    )
    def update_table(selected_tickers, time_window_days, display_options, full_data_clicks):
        ctx = dash.callback_context
        # If Show Full Data button was clicked most recently
        if ctx.triggered and ctx.triggered[0]['prop_id'].startswith('show-full-data-btn'):
            return dash_table.DataTable(
                id='full-data-table',
                columns=[{"name": i, "id": i} for i in FX_Swap_df.columns],
                data=FX_Swap_df.to_dict('records'),
                page_size=20,
                style_table={'overflowX': 'auto', 'maxHeight': '500px'},
                style_cell={'minWidth': '100px', 'whiteSpace': 'normal'},
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}
            )

        # Otherwise, if 'show_table' checked, show summary stats
        if 'show_table' in display_options:
            cutoff_date = max_date - timedelta(days=time_window_days)
            filtered_df = FX_Swap_df[
                (FX_Swap_df['date'] >= cutoff_date) &
                (FX_Swap_df['ticker'].isin(selected_tickers))
            ]
            if not filtered_df.empty:
                summary_df = (
                    filtered_df
                    .groupby(['ticker', 'bucket'])['volume']
                    .agg(['mean', 'min', 'max', 'std'])
                    .reset_index()
                    .round(2)
                )
                return html.Div([
                    html.H3("Summary Statistics"),
                    dash_table.DataTable(
                        id='data-table',
                        columns=[{"name": i, "id": i} for i in summary_df.columns],
                        data=summary_df.to_dict('records'),
                        style_table={'overflowX': 'auto'},
                        style_cell={
                            'minWidth': '100px', 'width': '150px', 'maxWidth': '200px',
                            'whiteSpace': 'normal'
                        },
                        style_header={
                            'backgroundColor': 'rgb(230, 230, 230)',
                            'fontWeight': 'bold'
                        },
                        style_data_conditional=[
                            {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}
                        ]
                    )
                ])

        # Otherwise, clear
        return None

    @app.callback(
        Output('volume-matrix-plot', 'figure'),
        [
            Input('ticker-selector', 'value'),
            Input('time-window-slider', 'value'),
            Input('display-options', 'value')
        ]
    )
    def update_plots(selected_tickers, time_window_days, display_options):
        cutoff_date = max_date - timedelta(days=time_window_days)
        filtered_df = (
            FX_Swap_df[
                (FX_Swap_df['date'] >= cutoff_date) &
                (FX_Swap_df['ticker'].isin(selected_tickers))
            ]
            .sort_values(['ticker', 'bucket', 'date'])
        )

        buckets = sorted(filtered_df['bucket'].unique())
        fig = make_subplots(
            rows=len(buckets),
            cols=len(selected_tickers),
            subplot_titles=selected_tickers,
            shared_yaxes='normalize' in display_options,
            vertical_spacing=0.05,
            horizontal_spacing=0.05
        )

        # Row labels
        for i, bucket in enumerate(buckets):
            fig.add_annotation(
                x=-0.07, y=1 - (i + 0.5)/len(buckets),
                xref="paper", yref="paper",
                text=bucket, showarrow=False, textangle=-90,
                font=dict(size=12), bgcolor='rgba(255,255,255,0.7)'
            )

        # Populate subplots
        for col_idx, ticker in enumerate(selected_tickers, start=1):
            for row_idx, bucket in enumerate(buckets, start=1):
                subset = filtered_df[
                    (filtered_df['ticker'] == ticker) &
                    (filtered_df['bucket'] == bucket)
                ]
                fig.add_trace(
                    go.Bar(
                        x=subset['date'],
                        y=subset['volume'],
                        showlegend=False, marker_color='#1f77b4', opacity=0.7
                    ),
                    row=row_idx, col=col_idx
                )
                if 'show_trend' in display_options and len(subset) > 1:
                    fig.add_trace(
                        go.Scatter(
                            x=subset['date'],
                            y=subset['volume'].rolling(7, min_periods=1).mean(),
                            mode='lines',
                            line=dict(color='red', width=2),
                            showlegend=False
                        ),
                        row=row_idx, col=col_idx
                    )

                fig.update_xaxes(
                    title_text="Date" if row_idx == len(buckets) else "",
                    row=row_idx, col=col_idx,
                    tickformat="%b %d", tickangle=45
                )
                if col_idx == 1:
                    fig.update_yaxes(
                        title_text="Volume", row=row_idx, col=col_idx
                    )

        fig.update_layout(
            height=200 * len(buckets),
            width=300 * max(len(selected_tickers), 1),
            title_text=f"FX Swap Volume Evolution (Last {time_window_days} Days)",
            margin=dict(l=100, r=50, b=100, t=50),
            hovermode="closest",
            plot_bgcolor='rgba(240,240,240,0.5)'
        )
        return fig

# ─── Entrypoint ───────────────────────────────────────────────────────────────
if __name__ == '__main__':
    app.layout = construct_layout()
    register_callbacks(app)
    app.run(debug=True, port=8051)
